## Table Of Contents
 1. [Caricamento dei dati](#load)<br>
     1.1 [Grafici di esempio](#esempi)
 2. [Calcolo delle variabili di riassunto](#variabili)
 3. [Analisi esplorativa con `PCA`, `ICA`, `t-SNE`](#esplorativa)
 4. [Modello multinomiale](#multinomiale)
 5. [Analisi discriminante](#lda-qda)
 6. [Alberi di regressione](#tree)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Funzioni base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import tqdm

# Font di LaTeX
# from matplotlib import rc

# Scikit-Learn
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.decomposition import PCA, FastICA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
# Funzioni custom
from funzioni import AbsMeanVarDeriv, Whiten,ScatterGroup, MatriceConfusione, indice_gini
from funzioni import tasso_errata_classificazione, grafico_metrica_iperparametro, grafico_metrica_iperparametri
from funzioni.grafici import grafico_importanza_variabili

# 1. Caricamento dei dati <a id=load> </a>

In [ ]:
PATH_DATA = './PhonePi/data/'
FIG_PATH = './figure/'
DIR = [os.path.join(PATH_DATA, o) for o in os.listdir(PATH_DATA) 
                    if os.path.isdir(os.path.join(PATH_DATA,o))]
tipo=[(dir.split("/")[-1]).split(".")[0] for dir in DIR]
tipo=[dir.split("-")[0] for dir in tipo]

In [ ]:
p = 150 # numero osservazioni per intervallo
nomi_colonna=["user","azione"]
nomi_colonna.extend(["a"+str(i) for i in range(p)])
nomi_colonna
X=pd.DataFrame(columns=nomi_colonna)

for i in tqdm.tqdm(range(len(DIR))):
    data = pd.read_csv(DIR[i] + "/accelerometer.txt", names = ["user", "type", "t", "ax", "ay", "az"]) # lettura dati
    data["t"] = data["t"] - data["t"].iloc[0] # t0 = 0
    data = data[(data["t"] > 7000) & (data["t"] < (data["t"].max()-7000))] # tolti i primi e ultimi 7 secondi
    data.reset_index(drop=True, inplace=True) # ripristinati gli indici da 0 in avanti
    data["a"] = (pd.to_numeric(data["ax"])**2 + pd.to_numeric(data["ay"])**2 + pd.to_numeric(data["az"])**2)**0.5 # accelerazione in modulo
    nome = [data.user[j] for j in range(0,len(data)-p, p)] # intervalli di dt*100ms
    tipologia=[tipo[i]]*len(nome)
    righe=[[nome[j],tipologia[j]] for j in range(len(nome))]
    [righe[j].extend(list(data.a[j*p:(j+1)*p])) for j in range(len(nome))]
    X=pd.concat([X,pd.DataFrame(righe,columns=nomi_colonna)],ignore_index=True) # ignore_index=T per avere indici consecutivi

## 1.1 Grafici di esempio <a id=esempi> </a>

In [ ]:
DIRM = ['./PhonePi/data/scale-martina',
        './PhonePi/data/salti-daniele',
        './PhonePi/data/shake-anna']

plt.figure(figsize=(13,5.5*len(DIRM)))
nrow = len(DIRM)
for i in tqdm.tqdm(range(len(DIRM))):
    data = pd.read_csv(DIRM[i] + "/accelerometer.txt", names = ["user", "type", "t", "ax", "ay", "az"])
    data["t"] = data["t"] - data["t"].iloc[0]
    data = data[(data["t"] > 7000) & (data["t"] < (data["t"].max()-7000))]
    data["a"] = (pd.to_numeric(data["ax"])**2 + pd.to_numeric(data["ay"])**2 + pd.to_numeric(data["az"])**2)**0.5 # accelerazione in modulo
    ax = plt.subplot(nrow, 1, i+1, ylim=(0,120), xlim = (10000,30000))
    ax.set_ylabel(r"$\|a\|\;(\mathrm{ m/s}^2)$", rotation=0)
    ax.yaxis.set_label_coords(-0.1,0.7)
    ax.set_xlabel(r"$t \;(\mathrm{ ms})$")
    ax.set_title((DIRM[i].split("/")[-1]).split("-")[0])
    plt.plot(data["t"],data["a"])
    plt.savefig(FIG_PATH+"espl.png", dpi=150)    
plt.show()

# 2. Calcolo delle variabili di riassunto <a id=variabili> </a>

In [ ]:
y = X.azione
X.drop("azione", axis=1, inplace=True)
Xnum = X.drop("user", axis=1)

In [ ]:
maxA = Xnum.max(1) # massimo accelerazione
MVDeriv = AbsMeanVarDeriv(Xnum, 10) # variazione media della derivata
Mean = Xnum.mean(axis=1)
Var = Xnum.var(axis=1)
Med = Xnum.median(axis=1)
Min = Xnum.min(axis=1)

In [ ]:
espl = pd.concat([maxA, MVDeriv, Mean, Var, Med, Min], axis=1)
espl.columns=["maxA", "MVDeriv", "meanA", "Var", "Med", "Min"]

In [ ]:
X.to_pickle("X-2s.pkl")
y.to_pickle("y-2s.pkl")
espl.to_pickle("espl.pkl")

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(espl, y, test_size=0.25, random_state=42)

# 3. Analisi esplorativa con `PCA`, `ICA`, `t-SNE` <a id=esplorativa> </a>

In [ ]:
# Sbiancamento dei dati
esplWh = Whiten().fit_transform(espl)

pca = PCA(n_components=2, random_state=42)
esplPCA = pca.fit_transform(esplWh)

ica = FastICA(n_components=2, random_state=42)
esplICA = ica.fit_transform(esplWh)

tsne = TSNE(n_components=2, random_state=42)
esplTSNE = tsne.fit_transform(esplWh)

In [ ]:
for title,dat in zip(["PCA","ICA","t-SNE"], [esplPCA, esplICA, esplTSNE]):
    fig, ax = ScatterGroup(pd.DataFrame(dat, columns=["Prima componente", "Seconda componente"]),
                       grp=y, palette="colorblind")
    fig.set_figwidth(11)
    fig.set_figheight(6)
    ax.set_title(title)
    plt.legend(bbox_to_anchor=(1,0.7))
    plt.savefig(FIG_PATH+title+".png", bbox_inches="tight", dpi=180)

# 4. Modello multinomiale <a id=multinomiale> </a>

In [ ]:
glmMult = LogisticRegression(penalty="l2", C=float("inf"), random_state=42, multi_class="multinomial",
                             solver="newton-cg", max_iter=1000)
fit = glmMult.fit(X_train, y_train)
y_pred = fit.predict(X_val)

acc_mn = 100*accuracy_score(y_val, y_pred)
print("LogisticRegression(multi_class=\"multinomial\"): {:.1f}% di accuratezza".format(acc_mn))
MatriceConfusione(y_val, y_pred)
plt.savefig(FIG_PATH+"confusionMatrix-Mn.png", dpi=300, bbox_inches="tight")

# 5. Analisi discriminante lineare e quadratica <a id=lda-qda> </a>

In [ ]:
lda = LinearDiscriminantAnalysis()
X_lda=lda.fit(X_train, y_train)
y_pred_lda = lda.predict(X_val)
acc_lda = 100*accuracy_score(y_val, y_pred_lda)
print("Accuratezza LDA: {:.1f}%".format(acc_lda))
MatriceConfusione(y_val, y_pred_lda,nome_immagine=FIG_PATH+"confusionMatrix-LDA")

In [ ]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)
y_pred_qda = qda.predict(X_val)
acc_qda = 100*accuracy_score(y_val, y_pred_qda)
print("Accuratezza QDA: {:.1f}%".format(acc_qda))
MatriceConfusione(y_val, y_pred_qda,nome_immagine=FIG_PATH+"confusionMatrix-QDA")

In [ ]:
np.bincount((y_pred_qda == "shake"))
sum(y_pred_qda == "shake")


# Da sviluppare, LDA e QDA con penalizzazione per gli shake

In [ ]:
weights = np.array(y_train.value_counts()/y_train.value_counts().sum())
lda = LinearDiscriminantAnalysis(priors=weights)
X_lda=lda.fit(X_train, y_train)
y_pred_lda = lda.predict(X_val)
lista_acc_lda = [100*accuracy_score(y_val, y_pred_lda),]
lista_veri_positivi_lda= [100*(confusion_matrix(y_val, y_pred_lda)[-1,-1]/confusion_matrix(y_val, y_pred_lda)[:,-1].sum()),]
lista_numero_sbagliati_lda= [confusion_matrix(y_val, y_pred_lda)[:-1,-1].sum(),]

In [ ]:
weights = y_train.value_counts()/y_train.value_counts().sum()

In [ ]:
weights

In [ ]:
lista_acc_lda = []
lista_veri_positivi_lda = []
lista_numero_sbagliati_lda = []
for i in np.arange(1,10,0.5):
    weights = np.array(y_train.value_counts()/y_train.value_counts().sum())
    weights[1] /= i
    weights = weights/weights.sum()
    lda = LinearDiscriminantAnalysis(priors=weights)
    X_lda=lda.fit(X_train, y_train)
    y_pred_lda = lda.predict(X_val)
    lista_acc_lda.append(100*accuracy_score(y_val, y_pred_lda))
    confMatrix = confusion_matrix(y_val, y_pred_lda, labels=lab)
    lista_veri_positivi_lda.append(100*(confMatrix[-1,-1]/confMatrix[:,-1].sum()))
    lista_numero_sbagliati_lda.append(confMatrix[:-1,-1].sum())
    #print(i)
    #print("Accuratezza LDA: {:.2f}%".format(acc_lda))
    #print("Tasso veri positivi LDA: {:.2f}%".format(veri_positivi))
    #print("Numero shake sbagliati LDA: {:.2f}".format(numero_sbagliati))
    #print("_______________")

In [ ]:
lab = y.unique()
lab.sort()
confusion_matrix(y_val, y_pred_lda, labels=lab)

In [ ]:
lista_veri_positivi_lda

In [ ]:
lab = y.unique()
lab.sort()
sns.heatmap(confusion_matrix(y_val, y_pred_lda), yticklabels=y.unique(), xticklabels=y.unique())
sns.heatmap(confusion_matrix(y_val, y_pred_lda), yticklabels=lab, xticklabels=lab)

In [ ]:
weights = np.array(y_train.value_counts()/y_train.value_counts().sum())
qda = QuadraticDiscriminantAnalysis(priors=weights)
qda.fit(X_train, y_train)
y_pred_qda = qda.predict(X_val)
lista_acc_qda = [100*accuracy_score(y_val, y_pred_qda),]
lista_veri_positivi_qda= [100*(confusion_matrix(y_val, y_pred_qda)[-1,-1]/confusion_matrix(y_val, y_pred_qda)[:,-1].sum()),]
lista_numero_sbagliati_qda= [confusion_matrix(y_val, y_pred_qda)[:-1,-1].sum(),]

In [ ]:
lista_acc_qda = []
lista_veri_positivi_qda = []
lista_numero_sbagliati_qda = []
for i in np.arange(1,10,0.5):
    weights = np.array(y_train.value_counts()/y_train.value_counts().sum())
    weights[1] /= i
    weights = weights/weights.sum()
    qda = QuadraticDiscriminantAnalysis(priors=weights)
    X_qda=qda.fit(X_train, y_train)
    y_pred_qda = qda.predict(X_val)
    lista_acc_qda.append(100*accuracy_score(y_val, y_pred_qda))
    confMatrix = confusion_matrix(y_val, y_pred_qda, labels=lab)
    lista_veri_positivi_qda.append(100*(confMatrix[-1,-1]/confMatrix[:,-1].sum()))
    lista_numero_sbagliati_qda.append(confMatrix[:-1,-1].sum())
    #print(i)
    #print("Accuratezza LDA: {:.2f}%".format(acc_lda))
    #print("Tasso veri positivi LDA: {:.2f}%".format(veri_positivi))
    #print("Numero shake sbagliati LDA: {:.2f}".format(numero_sbagliati))
    #print("_______________")

In [ ]:
lista_veri_positivi_qda

In [ ]:
# in pratica per pesi crescenti assegnati alla classe dello shake si nota che la percentuale di veri positivi aumenta
# e l'accuratezza ovviamente diminuisce. Se vogliamo che l'errore di classificare shake qualcosa che non è shake sia nullo
# (o <0.01) scegliamo il primo peso che mi porta ad avere veri_positivi uguale a 100 (o 99) in modo da scegliere 
# il modello con l'accuratezza migliore fra quelli che non sbagliano lo shake

plt.plot(lista_acc_lda, label="accuracy")
plt.plot(lista_veri_positivi_lda, label="veri positivi")
plt.legend()
#plt.plot(lista_numero_sbagliati_lda)
plt.title("LDA")
plt.show()

plt.plot(lista_acc_qda, label="accuracy")
plt.plot(lista_veri_positivi_qda, label="veri positivi")
plt.legend()
#plt.plot(lista_numero_sbagliati_qda)
plt.title("QDA")
plt.show()

Seleziono il peso da assegnare alla classe shake per avere un tasso di veri positivi del 99%

In [ ]:
lista_veri_positivi_lda=np.array(lista_veri_positivi_lda)
num_iterazione_lda=np.min(np.where(lista_veri_positivi_lda>99))
peso_shake_lda=np.arange(1.1,8,0.1)[num_iterazione_lda]
peso_shake_lda

In [ ]:
lista_veri_positivi_qda=np.array(lista_veri_positivi_qda)
num_iterazione_qda=np.min(np.where(lista_veri_positivi_qda>99))
peso_shake_qda=np.arange(1.1,8,0.1)[num_iterazione_qda]
peso_shake_qda

In [ ]:
weights = np.array(y_train.value_counts()/y_train.value_counts().sum())
weights[-1] /= peso_shake_lda
weights = weights/weights.sum()
lda = LinearDiscriminantAnalysis(priors=weights)
X_lda=lda.fit(X_train, y_train)
y_pred_lda = lda.predict(X_val)
acc_lda = 100*accuracy_score(y_val, y_pred_lda)
print("Accuratezza QDA: {:.1f}%".format(acc_lda))
MatriceConfusione(y_val, y_pred_lda,nome_immagine=FIG_PATH+"confusionMatrix-LDA-penalizzata")

In [ ]:
weights = np.array(y_train.value_counts()/y_train.value_counts().sum())
weights[-1] /= peso_shake_qda
weights = weights/weights.sum()
qda = QuadraticDiscriminantAnalysis(priors=weights)
qda.fit(X_train, y_train)
y_pred_qda = qda.predict(X_val)
acc_qda = 100*accuracy_score(y_val, y_pred_qda)
print("Accuratezza QDA: {:.1f}%".format(acc_qda))
MatriceConfusione(y_val, y_pred_qda,nome_immagine=FIG_PATH+"confusionMatrix-QDA-penalizzata")

# 6. Alberi di regressione <a id=tree> </a>

In [ ]:
# Albero stimato sul training, senza vincoli (albero completo)
dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_val)
acc_dtcFull = 100*accuracy_score(y_val, y_pred)

print("Accuratezza DecisionTreeClassifier(): {:.2f}%".format(acc_dtcFull))
MatriceConfusione(y_val, y_pred)
plt.show()

In [ ]:
maxDepth = dtc.tree_.max_depth
minObs = len(X_train) // 2
print("Profondità dell'albero allenato senza restrizioni: {}".format(maxDepth))
print("Massimo numero minimo di osservazioni in una foglia: {}".format(minObs))

In [ ]:
param_grid = ParameterGrid({
    'max_depth': np.arange(1, dtc.tree_.max_depth),
    'min_samples_leaf': 2 ** np.arange(int(np.log2(minObs) + 1)),
})
print(param_grid.param_grid)

In [ ]:
risultati = []

for params in tqdm.tqdm(param_grid):
    dtc = DecisionTreeClassifier(random_state=42, **params)
    dtc.fit(X_train, y_train)
    y_pred = dtc.predict(X_val)
    params["accuracy_score"] = accuracy_score(y_val, y_pred)
    risultati.append(params)

risultati = pd.DataFrame(risultati).sort_values(["accuracy_score", "max_depth"], ascending=[False, True])
risultati.reset_index(drop=True, inplace=True)
print("Primi 5:")
display(risultati.head())

print("Ultimi 5:")
risultati.tail()

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(121)
grafico_metrica_iperparametro(risultati, "max_depth", "accuracy_score", alpha=0.5)

plt.subplot(122)
grafico_metrica_iperparametro(risultati, "min_samples_leaf", "accuracy_score", alpha=0.5)
plt.xscale("log", basex=2)

plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

grafico_metrica_iperparametri(risultati, "max_depth", "min_samples_leaf", "accuracy_score")
plt.yscale("log", basey=2)
plt.savefig(FIG_PATH + "iperparametri-Tree.png", dpi=200)
plt.show()

In [ ]:
max_depth = risultati.loc[0, "max_depth"]
min_samples_leaf = risultati.loc[0, "min_samples_leaf"]

dtcTun = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=42)
dtcTun.fit(X_train, y_train)

y_pred = dtcTun.predict(X_val)
acc_dtcTun = 100*accuracy_score(y_val, y_pred)
print("profondità ottimale:",max_depth)
print("numero ottimale minimo di unità per foglia:",min_samples_leaf)

In [ ]:
print("Accuratezza DecisionTreeClassifier(): {:.1f}%".format(acc_dtcFull))
print("Accuratezza DecisionTreeClassifier(max_depth={}, min_samples_leaf={}): {:.1f}%".format(
    max_depth, min_samples_leaf, acc_dtcTun))
print(confusion_matrix(y_val, y_pred))

MatriceConfusione(y_val, y_pred)
plt.savefig(FIG_PATH + "confusionMatrix-Tree.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
importanze = dtcTun.feature_importances_
variabili = espl.columns

grafico_importanza_variabili(importanze, variabili)
plt.savefig(FIG_PATH + "importance-Tree.png", dpi=300, bbox_inches="tight")
plt.show()

# 7. Salvataggio dei risultati

## Tabella accuracy in file LaTeX

In [ ]:
tableEnvBegin = "\\begin{table}[H]\n\\centering"
tableEnvEnd = "\end{table}"

In [ ]:
accuracy = pd.DataFrame([["Multinomiale", acc_mn],
             ["LDA", acc_lda],
             ["QDA", acc_qda],
             ["Decision Tree", acc_dtcFull],
             ["Pruned Decision Tree", acc_dtcTun]], columns=["Modello", "Accuracy %"])

accuracy.sort_values("Accuracy %", inplace=True)
caption='\\caption{Accuratezza per i modelli adattati.}\n'

In [ ]:
with open("./relazione/tex/accuracy-table.tex", mode="w") as file:
    file.write(tableEnvBegin + caption + accuracy.to_latex(index=False, float_format="%.2f", column_format="cc") + tableEnvEnd)